In [ ]:
from shippy.db.sqldb import ShippySQLite
from shippy.data_ingest import ingest_functions
from shippy.kpi_calculators import calculator_functions
import pandas as pd

In [ ]:
db = ShippySQLite()

# Test Database and Ingest Functions.

In [ ]:
raw_messages = ingest_functions.raw_messages_csv_to_df()
db.write_df_to_table(raw_messages, 'raw_messages', if_exists='replace')

In [ ]:
pd.concat([raw_messages,db.read_table_to_df('raw_messages').set_index(['device_id', 'datetime'])]).drop_duplicates(keep=False).shape[0]

In [ ]:
weather_stations = ingest_functions.weather_data_json_to_df()
db.write_df_to_table(weather_stations, 'weather_stations', if_exists='replace')

In [ ]:
pd.concat([weather_stations,db.read_table_to_df('weather_stations').set_index('index')]).drop_duplicates(keep=False).shape[0]

In [ ]:
weather_data = ingest_functions.weather_data_json_normalized_to_df()
db.write_df_to_table(weather_data, 'weather_data', if_exists='replace')

In [ ]:
pd.concat([weather_data,db.read_table_to_df('weather_data').set_index('timestamp_utc')]).drop_duplicates(keep=False).shape[0]

In [ ]:
raw_messages_stations = ingest_functions.raw_messages_and_nearest_station_to_df()
db.write_df_to_table(raw_messages_stations, 'raw_messages_stations', if_exists='replace')

In [ ]:
pd.concat([raw_messages_stations,db.read_table_to_df('raw_messages_stations').set_index(['device_id', 'datetime'])]).drop_duplicates(keep=False).shape[0]

# Test KPI Calculator Functions.

In [ ]:
raw_messages_stations = db.read_table_to_df('raw_messages_stations', parse_dates=['datetime']).set_index(['device_id', 'datetime'])
weather_data = db.read_table_to_df('weather_data', parse_dates = ['timestamp_utc']).set_index('timestamp_utc')

In [ ]:
weather_data.index

In [ ]:
calculator_functions.num_ships_in_data(raw_messages_stations)

In [ ]:
calculator_functions.avg_speeds_for_date(raw_messages_stations)

In [ ]:
calculator_functions.wind_speed_max_min_for_ship(raw_messages_stations, weather_data)

In [ ]:
calculator_functions.full_weather_visual_for_ship(raw_messages_stations, weather_data).show()

In [ ]:
calculator_functions._merge_raw_messages_weather(raw_messages_stations, 
                                                 weather_data, 
                                                 device_id = 'st-1a2090',
                                                 date= pd.to_datetime('2019-02-13'))